# Object detection and motion detection using TensorFlow 2 and TensorFlow Lite

In this notebook we will go through the process of configuring TensorFlow 2 to process an mp4 video frame-by-frame and perform object detection, paired with motion detection. 

This will involve:

1. Cloning the TensorFlow Model Garden repo
2. Installing TensorFlow 2 to your Colab environment
3. Testing your environment with the pre-built TensorFlow Model Test's 
4. Downloading a pre-trained TensorFlow Lite model
5. Running this model against an mp4 video

If there are any issues with this notebook please contact james.milward@credera.co.uk


# Installation / Environment Configuration

First, install the required libraries. We'll start off by getting the latest TensorFlow Models.

**Note** the uninstall of Cython, it causes issues with TensorFlow packages

In [ ]:
# Clone the tensorflow models repository from GitHub and remove Cython (causes issues with Colab)
!git clone --depth 1 https://github.com/tensorflow/models tensorflow-models
!pip uninstall Cython -y

Next we'll install Tensorflow's object detection functionality by copying out the setup script and modifying it to support the latest version of TensorFlow that works with Colab (2.15.0)

In [2]:
# Copy setup files into models/research folder
%%bash
cd tensorflow-models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .

In [3]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.15.0 (latest as of Aug 2024)
import re
with open('/content/tensorflow-models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/tensorflow-models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.15.0', s)
    f.write(s)

In [ ]:
# Install TensorFlow specific version of pyyaml, install the object detection based on the modified setup.py and install TensorFlow
!pip install pyyaml==5.3
!pip install /content/tensorflow-models/research/
!pip install tensorflow==2.15.0

You may get errors or warnings with a modal window informing you that you need to restart. Ignore these errors, close the modal (don't restart) and move on.

Next, lets install the model builder to make sure everything is working. The model builder test script runs through a few tests of the EfficientDet object detection model to ensure TensorFlow is working correctly.

In [ ]:
# Run Model Bulider Test file, just to verify everything's working properly
!python /content/tensorflow-models/research/object_detection/builders/model_builder_tf2_test.py


# Setup pre-trained model

In [ ]:
# Clone the tensorflow models repository from GitHub and remove Cython (causes issues with Colab)
!git clone -b feat/colab-training --depth 1 https://github.com/Jimbwlah/tensorflow-ring-animal-detector

# Test TensorFlow Lite Model

We've now cloned a repo with a sample object detection model trained against animals. Let's get this setup in Python to execute object detection against a stock video.

Firstly we change directory -

In [7]:
cd /content/tensorflow-ring-animal-detector/src

/content/tensorflow-ring-animal-detector/src


Next we want to use the VideoStream class, which allows for frame by frame analysis (with a given framerate) of an mp4 video.

This script does the following -


* Takes a pre-trained model (PATH_TO_CKPT)
* Accesses a label map
* Instantiates the TensorFlow interpreter
* Goes frame by frame through a video performing motion detection and object detection



In [ ]:
import os
import cv2
import numpy as np
from datetime import datetime, timedelta
from tensorflow.lite.python.interpreter import Interpreter
from video_streaming.video_stream import VideoStream

import matplotlib
import matplotlib.pyplot as plt

# Path to .tflite file, which contains the model that is used for object detection
PATH_TO_CKPT = os.path.join('/content/tensorflow-ring-animal-detector/src/models/custom_trained/ssdmobilenet_v2_320/detect.tflite')

# Path to label map file
PATH_TO_LABELS = os.path.join('/content/tensorflow-ring-animal-detector/training-content/labelmap.txt')

# Video Resolutions
resolution_w = 640
resolution_h = 480

# Load the label map
with open(PATH_TO_LABELS, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

# Load the Tensorflow Lite model.
interpreter = Interpreter(model_path=PATH_TO_CKPT)
interpreter.allocate_tensors()

# Get and store/calculate TFLite model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
floating_model = (input_details[0]['dtype'] == np.float32)
input_mean = 127.5
input_std = 127.5
boxes_idx, classes_idx, scores_idx = 1, 3, 0 # This is a TF2 model

# Initialize frame rate calculation
frame_rate_calc = 1
freq = cv2.getTickFrequency()

# Initialize video stream
videostream = VideoStream(
    resolution=(int(resolution_w),int(resolution_h)),
    video_url_or_filename='/content/tensorflow-ring-animal-detector/src/test-video/badger-stock-video.mp4',
    framerate=10,
    video_file_analysis=True).start()
window_name = "Animal Object Detection"

# Max streaming time
time_to_stop_video = datetime.now() + timedelta(seconds=float(30))

# FPS control
fps_delay = datetime.now() + timedelta(milliseconds=100)

# Motion detection variables
first_frame = None
frame_diff = None
motion_contours = None

while True:

    # Start timer (for calculating frame rate)
    t1 = cv2.getTickCount()

    # Implement FPS delay
    if datetime.now() >= fps_delay:

        try:
            # Grab frame from video stream
            frame1 = videostream.read()
            if type(frame1) is not None:

                frame1 = cv2.resize(frame1, (resolution_w, resolution_h))

                # Acquire frame and resize to expected shape [1xHxWx3]
                frame = frame1.copy()
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_resized = cv2.resize(frame_rgb, (width, height))
                input_data = np.expand_dims(frame_resized, axis=0)

                #### Motion Detection ####
                # Thanks to https://towardsdatascience.com/image-analysis-for-beginners-creating-a-motion-detector-with-opencv-4ca6faba4b42
                # Prepare image; grayscale and blur
                prepared_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_BGR2GRAY)
                prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5,5), sigmaX=0)

                # Calculate difference and update previous frame
                if (first_frame is None):
                    first_frame = prepared_frame
                else:
                    frame_diff = cv2.absdiff(src1=first_frame, src2=prepared_frame) # calculate the dif
                    first_frame = prepared_frame # replace the first frame

                    # Dilute the image a bit to make differences more seeable; more suitable for contour detection
                    kernel = np.ones((5, 5))
                    frame_diff = cv2.dilate(frame_diff, kernel, 1)

                    # Only take different areas that are different enough (>10 / 255)
                    thresh_frame = cv2.threshold(src=frame_diff, thresh=25, maxval=255, type=cv2.THRESH_BINARY)[1]
                    plt.imshow(thresh_frame)
                    plt.show()

                    # Use motion contours when an object is detected and correlate
                    motion_contours, _ = cv2.findContours(image=thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)

                #### Object Detection ####
                # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
                if floating_model:
                    input_data = (np.float32(input_data) - input_mean) / input_std

                # Perform the actual detection by running the model with the image as input
                interpreter.set_tensor(input_details[0]['index'],input_data)
                interpreter.invoke()

                # Retrieve detection results
                boxes = interpreter.get_tensor(output_details[boxes_idx]['index'])[0] # Bounding box coordinates of detected objects
                classes = interpreter.get_tensor(output_details[classes_idx]['index'])[0] # Class indeqx of detected objects
                scores = interpreter.get_tensor(output_details[scores_idx]['index'])[0] # Confidence of detected objects

                # Loop over all detections and draw detection box if confidence is above minimum threshold (60%)
                for i in range(len(scores)):
                    if ((scores[i] > 0.6) and (scores[i] <= 1.0)):

                        # Get bounding box coordinates and draw box
                        # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
                        ymin = int(max(1,(boxes[i][0] * resolution_h)))
                        xmin = int(max(1,(boxes[i][1] * resolution_w)))
                        ymax = int(min(resolution_h,(boxes[i][2] * resolution_h)))
                        xmax = int(min(resolution_w,(boxes[i][3] * resolution_w)))
                        cv2.rectangle(frame, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

                        # Draw label
                        object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
                        label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
                        labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
                        label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
                        cv2.rectangle(frame, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
                        cv2.putText(frame, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

                        # Motion detection inside object detection - determine if it's near the object that's been detected.
                        # This improves object detection as it can mistakenly recognise stationary grass/leaves/patterns as animals
                        motion_near_object = False
                        search_hit = bool([ele for ele in ['fox', 'badger'] if(ele in object_name)])
                        if search_hit:
                            if motion_contours != None:
                                for contour in motion_contours:
                                    (x, y, w, h) = cv2.boundingRect(contour)

                                    # Calculate difference between coords of detected object and detected motion
                                    ymin_calc = abs(ymin-y)
                                    xmin_calc = abs(xmin-x)
                                    ymax_calc = abs(ymax-(y+h))
                                    xmax_calc = abs(xmax-(x+w))

                                    # Motion detected if difference is no lower than threshold (20% of resolution width)
                                    threshold = resolution_w/20
                                    if ymin_calc <= threshold and xmin_calc <= threshold and ymax_calc <= threshold and xmax_calc <= threshold:
                                        cv2.rectangle(img=frame, pt1=(x, y), pt2=(x + w, y + h), color=(255, 0, 0), thickness=2) # blue
                                        motion_near_object = True

                # Draw framerate in corner of frame
                cv2.putText(frame,'FPS: {0:.2f}'.format(frame_rate_calc),(30,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2,cv2.LINE_AA)

                # All the results have been drawn on the frame, so it's time to display it.
                plt.imshow(frame)
                plt.show()

                # Calculate framerate
                t2 = cv2.getTickCount()
                time1 = (t2-t1)/freq
                frame_rate_calc= 1/time1

                # Reset FPS delay
                fps_delay = datetime.now() + timedelta(milliseconds=100)

                if time_to_stop_video <= datetime.now():
                    break

        except Exception as inst:
            # Clean up and stop
            videostream.stop()

# Clean up
videostream.stop()

# Finally

You should now have seen object detection and motion detection running against the supplied stock footage of a badger! 

Some notes

* The Matlab plt. funtionality was used here instead of cv2.imshow() as the cv2 window functionality for showing images is not supported by Colab
* There may have been a few bugs in the frame-by-frame loop - this is a work in progress and I will be fixing this
* Further reading on how this works is contained within the tensorflow-ring-animal-detector repo which provides instructions on running this locally in vscode and training your own model